# 🚢 Análise Preditiva de Sobrevivência no Titanic

Este projeto utiliza o famoso dataset "Titanic: Machine Learning from Disaster" do Kaggle para construir um modelo preditivo. O objetivo é responder à pergunta: **"Que tipo de pessoa tinha maior probabilidade de sobreviver ao desastre?"**

Para isso, realizaremos uma análise exploratória dos dados, seguida pelo pré-processamento e treinamento de um modelo de classificação (Random Forest) para prever o desfecho de cada passageiro.

# 1. CONFIGURAÇÃO INICIAL E IMPORTAÇÃO DE BIBLIOTECAS

In [ ]:
#@title Instala a biblioteca para profiling de dados de forma silenciosa
%pip install -q ydata-profiling

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.3/399.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 679.7/679.7 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 1.3 MB/s eta 0:00:00


In [ ]:
#@title Importa as bibliotecas necessárias
!pip install ydata-profiling
import pandas as pd
import numpy as np
from ydata_profiling import ProfileReport
from google.colab import drive
from sklearn.ensemble import RandomForestClassifier

Abaixo Colocaremos uma configuração para exibir o resultado de alguns comandos na mesma linha.

In [ ]:
# Ajusta as configurações de exibição do Pandas
pd.set_option('display.max_columns', None)  # Mostra todas as colunas
pd.set_option('display.width', 1000)        # Aumenta a largura máxima da linha antes de quebrar

In [ ]:
#@title Monta o Google Drive para permitir o acesso aos arquivos
drive.mount('/content/drive')

Mounted at /content/drive


# 2. CARREGAMENTO E ANÁLISE INICIAL DOS DADOS

treino.csv (train) → contém os dados com a resposta (quem sobreviveu ou não).

teste.csv (test) → contém dados sem a resposta. É com esse que você vai prever.

In [ ]:
#@title Define os caminhos para os arquivos no seu Google Drive
train = pd.read_csv("/content/drive/MyDrive/Dados/Titanic/train.csv")
test = pd.read_csv("/content/drive/MyDrive/Dados/Titanic/test.csv")

In [ ]:
#@title Geração do Profile para um primeiro overview dos dados

#Gera um html pronto já com algumas informações importantes para iniciar a análise exploratória

#Salva o html do Profile na pasta Dados/Titanic do Drive
profile = ProfileReport(test, title="primeiras visões dos dados")
profile.to_file("/content/drive/MyDrive/Dados/Titanic/PrimeirasImpressões_Titanic.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]


100%|██████████| 11/11 [00:00<00:00, 51.29it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
#@title Exibe as primeiras linhas para verificação
print("--- Amostra do Dataset de Treino ---")
print(train.head(3))
print("\n--- Amostra do Dataset de Teste ---")
print(test.head(3))

--- Amostra do Dataset de Treino ---
   PassengerId  Survived  Pclass                                               Name     Sex   Age  SibSp  Parch            Ticket     Fare Cabin Embarked
0            1         0       3                            Braund, Mr. Owen Harris    male  22.0      1      0         A/5 21171   7.2500   NaN        S
1            2         1       1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1      0          PC 17599  71.2833   C85        C
2            3         1       3                             Heikkinen, Miss. Laina  female  26.0      0      0  STON/O2. 3101282   7.9250   NaN        S

--- Amostra do Dataset de Teste ---
   PassengerId  Pclass                              Name     Sex   Age  SibSp  Parch  Ticket    Fare Cabin Embarked
0          892       3                  Kelly, Mr. James    male  34.5      0      0  330911  7.8292   NaN        Q
1          893       3  Wilkes, Mrs. James (Ellen Needs)  female  47.0      1 

In [ ]:
# Exibe as dimensões dos datasets
print("\n--- Dimensões (Linhas, Colunas) ---")
print("Treino:", train.shape)
print("Teste:", test.shape)


--- Dimensões (Linhas, Colunas) ---
Treino: (891, 12)
Teste: (418, 11)


In [ ]:
print('Informações gerais da base train')
train.info()
print("\n")
print('Informações gerais da base test')
test.info()

Informações gerais da base train
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


Informações gerais da base test
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       ------------

In [ ]:
# Verifica a contagem de valores nulos antes de qualquer tratamento
print("\n--- Contagem de Nulos (Antes do Tratamento) ---")
print("Nulos em Treino:\n", train.isnull().sum())
print("\nNulos em Teste:\n", test.isnull().sum())


--- Contagem de Nulos (Antes do Tratamento) ---
Nulos em Treino:
 PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

Nulos em Teste:
 PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64


# 3. PRÉ-PROCESSAMENTO E LIMPEZA DOS DADOS

In [ ]:
train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [ ]:
# Remove colunas consideradas irrelevantes para este modelo inicial
train.drop(columns=['Name', 'Ticket', 'Cabin'], inplace=True)
test.drop(columns=['Name', 'Ticket', 'Cabin'], inplace=True)
print("\nColunas 'Name', 'Ticket' e 'Cabin' foram removidas.")


Colunas 'Name', 'Ticket' e 'Cabin' foram removidas.


### 3.4 Análise Específica da Coluna 'Sex'
Vamos investigar a coluna 'Sex' para entender sua distribuição, os valores únicos que ela contém e garantir que os dados estão consistentes antes de qualquer transformação.

In [ ]:
#@title Verificação e Contagem de Valores na coluna 'Sex'

# Boa prática: converter para minúsculas para garantir consistência (ex: 'Male' e 'male' seriam tratados como iguais)
# Neste dataset específico não é necessário, mas é um ótimo hábito.
# train['Sex'] = train['Sex'].str.lower()

print("Valores únicos na coluna 'Sex':")
print(train['Sex'].unique())

print("\n-----------------------------------\n")

print("Contagem de valores (Gênero):")
print(train['Sex'].value_counts())

# Opcional: Normalizar para ver a porcentagem
print("\nPorcentagem de valores (Gênero):")
print(train['Sex'].value_counts(normalize=True) * 100)

Valores únicos na coluna 'Sex':
['male' 'female']

-----------------------------------

Contagem de valores (Gênero):
Sex
male      577
female    314
Name: count, dtype: int64

Porcentagem de valores (Gênero):
Sex
male      64.758698
female    35.241302
Name: proportion, dtype: float64


Tratamento de Valores Nulos

In [ ]:
# Coluna 'Age': Preenche os valores nulos com a MÉDIA de idade do conjunto de TREINO
media_idade_train = train['Age'].mean()
train['Age'] = train['Age'].fillna(media_idade_train)
test['Age'] = test['Age'].fillna(media_idade_train)

# Coluna 'Embarked': Preenche os valores nulos com a MODA (valor mais comum) do conjunto de TREINO
moda_embarked_train = train['Embarked'].mode()[0]
train['Embarked'] = train['Embarked'].fillna(moda_embarked_train)
test['Embarked'] = test['Embarked'].fillna(moda_embarked_train)

# Coluna 'Fare': Preenche o valor nulo do conjunto de TESTE com a MEDIANA do conjunto de TREINO
mediana_fare_train = train['Fare'].median()
test['Fare'] = test['Fare'].fillna(mediana_fare_train)

# 4. FEATURE ENGINEERING (TRANSFORMAÇÃO DE DADOS)

In [ ]:
# Converte a coluna 'Sex' para formato numérico (male: 0, female: 1)
train['Sex'] = train['Sex'].map({'male': 0, 'female': 1})
test['Sex'] = test['Sex'].map({'male': 0, 'female': 1})

# Converte a coluna 'Embarked' usando One-Hot Encoding
# drop_first=True remove a primeira categoria para evitar multicolinearidade
train = pd.get_dummies(train, columns=['Embarked'], drop_first=True)
test = pd.get_dummies(test, columns=['Embarked'], drop_first=True)

print("Colunas categóricas convertidas para formato numérico.")

Colunas categóricas convertidas para formato numérico.


In [ ]:
# Verifica se todos os nulos foram tratados
print("\nSoma total de nulos no treino (após tratamento):", train.isnull().sum().sum())
print("Soma total de nulos no teste (após tratamento):", test.isnull().sum().sum())



Soma total de nulos no treino (após tratamento): 0
Soma total de nulos no teste (após tratamento): 0


# 5. TREINAMENTO DO MODELO RANDOM FOREST

In [ ]:
# Define as colunas que serão usadas como features (variáveis de entrada)
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked_Q', 'Embarked_S']

In [ ]:
# Separa os dados em X (features) e y (variável alvo - 'Survived')
X_train = train[features]
y_train = train['Survived']
X_test = test[features]


In [ ]:
# Garante que as colunas de treino e teste estejam na mesma ordem
X_test = X_test[X_train.columns]

In [ ]:
# Instancia o modelo RandomForestClassifier
model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42, n_jobs=-1)


In [ ]:
# Treina o modelo com os dados de treino
model.fit(X_train, y_train)
print(f"\nModelo RandomForest treinado! Acurácia nos dados de treino: {model.score(X_train, y_train) * 100:.2f}%")


Modelo RandomForest treinado! Acurácia nos dados de treino: 85.19%


# 6. PREVISÃO E CRIAÇÃO DO ARQUIVO DE SUBMISSÃO

In [ ]:
# Faz as previsões no conjunto de teste
predictions = model.predict(X_test)

In [ ]:
# Cria um novo DataFrame para o arquivo de submissão
submission = pd.DataFrame({
    'PassengerId': test['PassengerId'],
    'Survived': predictions
})

In [ ]:
# Define o caminho e salva o arquivo de submissão em formato .csv
submission_path = "/content/drive/MyDrive/Dados/Titanic/submission_random_forest.csv"
submission.to_csv(submission_path, index=False)

print(f"\nArquivo de submissão foi salvo com sucesso em: {submission_path}")
print("\nAmostra do arquivo de submissão:")
print(submission.head())


Arquivo de submissão foi salvo com sucesso em: /content/drive/MyDrive/Dados/Titanic/submission_random_forest.csv

Amostra do arquivo de submissão:
   PassengerId  Survived
0          892         0
1          893         0
2          894         0
3          895         0
4          896         1
